In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50, DenseNet121, MobileNet, MobileNetV2, VGG16
from tensorflow.keras.optimizers import AdamW, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import math
import itertools
import cv2

# Data loading
data_dir = r"/kaggle/input/eye-diseases-classification/dataset"
filepaths = []
labels = []
folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)

print("Class distribution:")
print(df['labels'].value_counts())
print(f"Number of unique classes: {df['labels'].nunique()}")

# Split data
train_df, dummy_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(dummy_df, train_size=0.6, shuffle=True, random_state=42)

print(f"Train samples: {len(train_df)}")
print(f"Valid samples: {len(valid_df)}")
print(f"Test samples: {len(test_df)}")

# Create data generators
batch_size = 32
img_size = (224, 224)
channels = 3

tr_gen = ImageDataGenerator(rescale=1./255)
ts_gen = ImageDataGenerator(rescale=1./255)

train_gen = tr_gen.flow_from_dataframe(
    train_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size
)
valid_gen = ts_gen.flow_from_dataframe(
    valid_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size
)
test_gen = ts_gen.flow_from_dataframe(
    test_df, x_col='filepaths', y_col='labels', 
    target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=False, batch_size=batch_size
)

# Get number of classes
num_classes = 4
class_names = list(train_gen.class_indices.keys())
print(f"Classes: {class_names}")

# Multiscale Attention Layer for Eye Disease Classifier
class MultiscaleAttentionLayer(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(MultiscaleAttentionLayer, self).__init__(**kwargs)
        self.filters = filters
        
    def build(self, input_shape):
        # Channel attention
        self.channel_attention = keras.Sequential([
            layers.GlobalAveragePooling2D(),
            layers.Dense(self.filters // 8, activation='relu'),
            layers.Dense(self.filters, activation='sigmoid'),
            layers.Reshape((1, 1, self.filters))
        ])
        
        # Spatial attention
        self.spatial_attention = keras.Sequential([
            layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')
        ])
        
        super(MultiscaleAttentionLayer, self).build(input_shape)
    
    def call(self, inputs):
        # Channel attention
        channel_att = self.channel_attention(inputs)
        x = layers.Multiply()([inputs, channel_att])
        
        # Spatial attention
        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
        spatial_input = layers.Concatenate()([avg_pool, max_pool])
        spatial_att = self.spatial_attention(spatial_input)
        x = layers.Multiply()([x, spatial_att])
        
        return x
    
    def get_config(self):
        config = super(MultiscaleAttentionLayer, self).get_config()
        config.update({'filters': self.filters})
        return config

# Eye Disease Classifier
def create_eye_disease_classifier(num_classes):
    """
    Create Eye Disease Classifier with VGG16 base and multiscale attention
    """
    try:
        # Load pre-trained VGG16
        base_model = VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        
        # Freeze early layers, unfreeze last few blocks for fine-tuning
        for layer in base_model.layers[:-4]:
            layer.trainable = False
        for layer in base_model.layers[-4:]:
            layer.trainable = True
            
        # Build the complete model
        inputs = keras.Input(shape=(224, 224, 3), name='input_image')
        
        # VGG16 feature extraction
        x = base_model(inputs, training=False)
        
        # Get the number of filters from VGG16 output
        vgg_output_filters = x.shape[-1]
        
        # Multiscale attention mechanism
        x = MultiscaleAttentionLayer(filters=vgg_output_filters, name='multiscale_attention')(x)
        
        # Global Average Pooling
        x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
        
        # Dropout for regularization
        x = layers.Dropout(0.5, name='dropout_1')(x)
        
        # Dense layer with batch normalization
        x = layers.Dense(256, activation='relu', name='dense_1')(x)
        x = layers.BatchNormalization(name='batch_norm')(x)
        x = layers.Dropout(0.25, name='dropout_2')(x)
        
        # Output layer with softmax activation
        predictions = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
        
        # Create the model
        model = keras.Model(inputs, predictions, name='EyeDiseaseClassifier')
        
        print(f"Eye Disease Classifier built successfully with {model.count_params()} parameters")
        return model
        
    except Exception as e:
        print(f"Error building model: {str(e)}")
        print("Building simplified model...")
        return create_eye_disease_classifier_simplified(num_classes)

def create_eye_disease_classifier_simplified(num_classes):
    """
    Build a simplified model without custom layers as fallback
    """
    # Load pre-trained VGG16
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze early layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True
        
    # Build simplified model
    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    
    # Simple attention using Global Average Pooling and Dense layers
    gap = layers.GlobalAveragePooling2D()(x)
    attention_weights = layers.Dense(x.shape[-1], activation='sigmoid')(gap)
    attention_weights = layers.Reshape((1, 1, x.shape[-1]))(attention_weights)
    x = layers.Multiply()([x, attention_weights])
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.25)(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, predictions, name='EyeDiseaseClassifier_Simplified')
    
    print(f"Simplified Eye Disease Classifier built successfully with {model.count_params()} parameters")
    return model

# ViT Small Implementation
class PatchEmbedding(layers.Layer):
    def __init__(self, patch_size=16, embed_dim=384):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.projection = layers.Conv2D(
            embed_dim, kernel_size=patch_size, strides=patch_size, padding='valid'
        )
        
    def call(self, x):
        batch_size = tf.shape(x)[0]
        patches = self.projection(x)
        patches = tf.reshape(patches, [batch_size, -1, self.embed_dim])
        return patches

class MultiHeadAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.qkv = layers.Dense(embed_dim * 3)
        self.projection = layers.Dense(embed_dim)
        
    def call(self, x):
        batch_size, seq_len, embed_dim = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        
        qkv = self.qkv(x)
        qkv = tf.reshape(qkv, [batch_size, seq_len, 3, self.num_heads, self.head_dim])
        qkv = tf.transpose(qkv, [2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        scale = tf.cast(self.head_dim, tf.float32) ** -0.5
        attn = tf.matmul(q, k, transpose_b=True) * scale
        attn = tf.nn.softmax(attn, axis=-1)
        
        out = tf.matmul(attn, v)
        out = tf.transpose(out, [0, 2, 1, 3])
        out = tf.reshape(out, [batch_size, seq_len, embed_dim])
        
        return self.projection(out)

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.dropout1 = layers.Dropout(dropout)
        
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = keras.Sequential([
            layers.Dense(mlp_hidden_dim, activation='gelu'),
            layers.Dropout(dropout),
            layers.Dense(embed_dim),
            layers.Dropout(dropout)
        ])
        
    def call(self, x, training=None):
        attn_out = self.attn(self.norm1(x))
        x = x + self.dropout1(attn_out, training=training)
        
        mlp_out = self.mlp(self.norm2(x), training=training)
        x = x + mlp_out
        
        return x

class VisionTransformerSmall(keras.Model):
    def __init__(self, 
                 img_size=224,
                 patch_size=16, 
                 num_classes=4,
                 embed_dim=384,
                 depth=12,
                 num_heads=6,
                 mlp_ratio=4.0,
                 dropout=0.1):
        super().__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        self.patch_embed = PatchEmbedding(patch_size, embed_dim)
        
        self.class_token = self.add_weight(
            shape=(1, 1, embed_dim),
            initializer='random_normal',
            trainable=True,
            name='class_token'
        )
        
        self.pos_embed = self.add_weight(
            shape=(1, self.num_patches + 1, embed_dim),
            initializer='random_normal',
            trainable=True,
            name='pos_embed'
        )
        
        self.dropout = layers.Dropout(dropout)
        
        self.blocks = [
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ]
        
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        self.head = layers.Dense(num_classes)
        
    def call(self, x, training=None):
        batch_size = tf.shape(x)[0]
        
        x = self.patch_embed(x)
        
        class_tokens = tf.broadcast_to(self.class_token, [batch_size, 1, tf.shape(self.class_token)[-1]])
        x = tf.concat([class_tokens, x], axis=1)
        
        x = x + self.pos_embed
        x = self.dropout(x, training=training)
        
        for block in self.blocks:
            x = block(x, training=training)
            
        x = self.norm(x)
        
        class_token_final = x[:, 0]
        return self.head(class_token_final)

def create_vit_small(num_classes):
    return VisionTransformerSmall(
        img_size=224,
        patch_size=16,
        num_classes=num_classes,
        embed_dim=384,
        depth=12,
        num_heads=6,
        mlp_ratio=4.0,
        dropout=0.1
    )

# Other model creation functions
def create_efficientnet(num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_mobilenet_v2(num_classes):
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_vgg16(num_classes):
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_denseNet201(num_classes):
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_mobilenet_v2(num_classes):
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# GradCAM implementation
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate GradCAM heatmap"""
    try:
        # Create a model that maps the input image to the activations of the last conv layer
        grad_model = tf.keras.models.Model(
            [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
        )
        
        # Compute the gradient of the top predicted class
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]
        
        # Gradient of the output neuron with regard to the output feature map
        grads = tape.gradient(class_channel, last_conv_layer_output)
        
        # Mean intensity of the gradient over a specific feature map channel
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Multiply each channel by "how important this channel is"
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        
        # Normalize the heatmap between 0 & 1
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        return heatmap.numpy()
    except Exception as e:
        print(f"GradCAM error: {e}")
        return None

# Fixed GradCAM implementation with proper layer detection

def get_last_conv_layer_name(model, model_name):
    """Enhanced function to get the last convolutional layer name"""
    conv_layers = []
    
    def collect_conv_layers(layer_or_model, prefix=""):
        """Recursively collect all conv layer names"""
        if hasattr(layer_or_model, 'layers'):
            for layer in layer_or_model.layers:
                layer_name = f"{prefix}{layer.name}" if prefix else layer.name
                
                # Check if it's a conv layer
                if any(conv_type in layer.__class__.__name__.lower() for conv_type in ['conv2d', 'separableconv2d', 'depthwiseconv2d']):
                    conv_layers.append(layer_name)
                
                # Recursively check sublayers
                collect_conv_layers(layer, prefix=f"{layer_name}/")
    
    # Start collection from the model
    collect_conv_layers(model)
    
    # If no conv layers found directly, try to access base model
    if not conv_layers:
        try:
            # For Sequential models
            if hasattr(model, 'layers') and len(model.layers) > 0:
                first_layer = model.layers[0]
                if hasattr(first_layer, 'layers'):
                    collect_conv_layers(first_layer)
        except:
            pass
    
    # Print available conv layers for debugging
    print(f"Available conv layers in {model_name}: {conv_layers[-5:]}")  # Show last 5
    
    # Return the last conv layer found
    return conv_layers[-1] if conv_layers else None

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Enhanced GradCAM with better error handling"""
    try:
        # Handle nested layer names (e.g., "base_model/block5_conv3")
        if '/' in last_conv_layer_name:
            layer_parts = last_conv_layer_name.split('/')
            current_layer = model
            
            # Navigate through nested layers
            for part in layer_parts:
                if hasattr(current_layer, 'get_layer'):
                    current_layer = current_layer.get_layer(part)
                elif hasattr(current_layer, 'layers'):
                    # Find layer by name in layers list
                    found = False
                    for layer in current_layer.layers:
                        if layer.name == part:
                            current_layer = layer
                            found = True
                            break
                    if not found:
                        raise ValueError(f"Layer {part} not found")
                else:
                    raise ValueError(f"Cannot navigate to layer {part}")
            
            target_layer = current_layer
        else:
            # Simple case - direct layer access
            target_layer = model.get_layer(last_conv_layer_name)
        
        # Create gradient model
        grad_model = tf.keras.models.Model(
            [model.inputs], 
            [target_layer.output, model.output]
        )
        
        # Compute gradients
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]
        
        # Get gradients
        grads = tape.gradient(class_channel, last_conv_layer_output)
        
        # Handle None gradients
        if grads is None:
            print(f"Warning: No gradients computed for layer {last_conv_layer_name}")
            return None
        
        # Compute importance weights
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Generate heatmap
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        
        # Normalize heatmap
        heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-10)
        return heatmap.numpy()
        
    except Exception as e:
        print(f"GradCAM failed for {last_conv_layer_name}: {str(e)}")
        return None

def plot_gradcam_examples_fixed(model, test_gen, class_names, model_name, num_examples=4):
    """Fixed GradCAM plotting with fallback strategies"""
    print(f"\nGenerating GradCAM for {model_name}...")
    
    # Get last conv layer with enhanced detection
    last_conv_layer = get_last_conv_layer_name(model, model_name)
    
    if not last_conv_layer:
        print(f"No convolutional layers found in {model_name} - skipping GradCAM")
        return
    
    print(f"Using layer: {last_conv_layer}")
    
    # Try alternative layers if the first one fails
    alternative_layers = []
    if 'vgg16' in last_conv_layer.lower():
        alternative_layers = ['block5_conv3', 'block5_conv2', 'block4_conv3']
    elif 'efficientnet' in model_name.lower():
        alternative_layers = ['top_activation', 'top_conv', 'block7a_expand_activation']
    elif 'mobilenet' in model_name.lower():
        alternative_layers = ['out_relu', 'Conv_1', 'block_16_depthwise']
    
    successful_layer = None
    test_heatmap = None
    
    # Test the primary layer
    test_gen.reset()
    test_images, test_labels = next(iter(test_gen))
    test_img = np.expand_dims(test_images[0], axis=0)
    
    test_heatmap = make_gradcam_heatmap(test_img, model, last_conv_layer, pred_index=0)
    if test_heatmap is not None:
        successful_layer = last_conv_layer
    else:
        # Try alternative layers
        for alt_layer in alternative_layers:
            try:
                test_heatmap = make_gradcam_heatmap(test_img, model, alt_layer, pred_index=0)
                if test_heatmap is not None:
                    successful_layer = alt_layer
                    print(f"Using alternative layer: {alt_layer}")
                    break
            except:
                continue
    
    if successful_layer is None:
        print(f"All GradCAM attempts failed for {model_name}")
        return
    
    # Plot GradCAM examples
    plt.figure(figsize=(20, 10))
    plt.suptitle(f'GradCAM Visualizations - {model_name} (Layer: {successful_layer})', 
                 fontsize=16, fontweight='bold')
    
    test_gen.reset()
    images, labels = next(iter(test_gen))
    
    for i in range(min(num_examples, len(images))):
        img_array = np.expand_dims(images[i], axis=0)
        
        # Make prediction
        preds = model.predict(img_array, verbose=0)
        pred_class = np.argmax(preds[0])
        true_class = np.argmax(labels[i])
        confidence = np.max(preds[0])
        
        # Generate heatmap
        heatmap = make_gradcam_heatmap(img_array, model, successful_layer, pred_class)
        
        # Plot original
        plt.subplot(2, num_examples, i + 1)
        plt.imshow(images[i])
        plt.title(f'Original\nTrue: {class_names[true_class]}', fontsize=10)
        plt.axis('off')
        
        # Plot GradCAM
        plt.subplot(2, num_examples, i + 1 + num_examples)
        plt.imshow(images[i])
        if heatmap is not None:
            # Resize heatmap to match image size
            heatmap_resized = cv2.resize(heatmap, (224, 224))
            plt.imshow(heatmap_resized, alpha=0.4, cmap='jet')
            status = "Success"
        else:
            status = "Failed"
        
        plt.title(f'GradCAM ({status})\nPred: {class_names[pred_class]}\nConf: {confidence:.3f}', 
                 fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def debug_model_structure(model, model_name):
    """Debug function to understand model structure"""
    print(f"\n=== MODEL STRUCTURE DEBUG: {model_name} ===")
    
    def print_layers(layer_or_model, indent=0):
        prefix = "  " * indent
        if hasattr(layer_or_model, 'layers'):
            print(f"{prefix}{layer_or_model.__class__.__name__}: {getattr(layer_or_model, 'name', 'unnamed')}")
            for i, layer in enumerate(layer_or_model.layers):
                print(f"{prefix}├─ [{i}] {layer.__class__.__name__}: {layer.name}")
                if hasattr(layer, 'layers') and len(layer.layers) > 0:
                    print_layers(layer, indent + 1)
        else:
            print(f"{prefix}{layer_or_model.__class__.__name__}: {getattr(layer_or_model, 'name', 'unnamed')}")
    
    print_layers(model)
    print("=" * 50)

# Modified plotting function call in main training loop
def plot_gradcam_examples(model, test_gen, class_names, model_name, num_examples=4):
    """Wrapper function that includes debugging"""
    # Debug model structure first
    debug_model_structure(model, model_name)
    
    # Use the fixed GradCAM implementation
    plot_gradcam_examples_fixed(model, test_gen, class_names, model_name, num_examples)

# Additional helper function to manually specify conv layer names
def get_manual_conv_layer_mapping():
    """Manual mapping for problematic models"""
    return {
        'EfficientNet-B0': ['efficientnetb0/block6a_expand_conv', 'efficientnetb0/top_conv'],
        'MobileNet-V2': ['mobilenetv2_1.00_224/block_16_depthwise', 'mobilenetv2_1.00_224/Conv_1'],
        'VGG16': ['vgg16/block5_conv3', 'vgg16/block5_conv2'],
        'DenseNet201': ['densenet201/conv5_block32_2_conv', 'densenet201/conv5_block32_concat'],
        'Eye Disease Classifier': ['vgg16/block5_conv3', 'multiscale_attention']
    }
# Enhanced visualization functions
def plot_clean_training_history(history, model_name):
    """Plot clean training history with 4-digit precision"""
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
    # Find best epochs
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'
    
    # Plot training history
    plt.figure(figsize=(15, 8))
    plt.style.use('fivethirtyeight')
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, label='Training loss', linewidth=2)
    plt.plot(Epochs, val_loss, label='Validation loss', linewidth=2)
    plt.scatter(index_loss + 1, val_lowest, s=150, c='blue', label=loss_label, zorder=5)
    plt.title(f'{model_name} - Training and Validation Loss', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label='Training Accuracy', linewidth=2)
    plt.plot(Epochs, val_acc, 'g', label='Validation Accuracy', linewidth=2)
    plt.scatter(index_acc + 1, acc_highest, s=150, c='blue', label=acc_label, zorder=5)
    plt.title(f'{model_name} - Training and Validation Accuracy', fontsize=14, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print best epoch information with 4-digit precision
    print(f"\n{model_name} Training Summary:")
    print(f"  Best Validation Loss: {val_lowest:.4f} at epoch {index_loss + 1}")
    print(f"  Best Validation Accuracy: {acc_highest:.4f} at epoch {index_acc + 1}")
    print(f"  Final Training Accuracy: {tr_acc[-1]:.4f}")
    print(f"  Final Validation Accuracy: {val_acc[-1]:.4f}")
    print(f"  Overfitting (Train-Val): {tr_acc[-1] - val_acc[-1]:.4f}")

def plot_confusion_matrix(y_true, y_pred, class_names, model_name):
    """Plot confusion matrix with 4-digit precision"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    plt.style.use('default')
    
    # Calculate percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create combined annotations (count and percentage)
    annotations = []
    for i in range(cm.shape[0]):
        row = []
        for j in range(cm.shape[1]):
            row.append(f'{cm[i,j]}\n({cm_percent[i,j]:.4f})')
        annotations.append(row)
    
    sns.heatmap(cm, annot=annotations, fmt='', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix - {model_name}\n(Count and Proportion)', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    return cm

def generate_classification_report_4digits(y_true, y_pred, class_names, model_name):
    """Generate detailed classification report with 4-digit precision"""
    report = classification_report(y_true, y_pred, target_names=class_names, 
                                 digits=4, output_dict=True)
    
    print(f"\n{'='*80}")
    print(f"CLASSIFICATION REPORT - {model_name}")
    print(f"{'='*80}")
    
    # Print header
    print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print("-" * 80)
    
    # Print per-class metrics
    for class_name in class_names:
        metrics = report[class_name]
        print(f"{class_name:<15} "
              f"{metrics['precision']:<12.4f} "
              f"{metrics['recall']:<12.4f} "
              f"{metrics['f1-score']:<12.4f} "
              f"{int(metrics['support']):<10}")
    
    print("-" * 80)
    
    # Print averages
    for avg_type in ['macro avg', 'weighted avg']:
        metrics = report[avg_type]
        print(f"{avg_type:<15} "
              f"{metrics['precision']:<12.4f} "
              f"{metrics['recall']:<12.4f} "
              f"{metrics['f1-score']:<12.4f} "
              f"{int(metrics['support']):<10}")
    
    print(f"\nAccuracy: {report['accuracy']:.4f}")
    print(f"{'='*80}")
    
    return report

def compile_and_train_model(model, model_name, train_gen, valid_gen, epochs=40):
    """Compile and train model with fixed callbacks (no ReduceLROnPlateau)"""
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    
    # Use fixed learning rate instead of schedule to avoid callback conflicts
    if 'Eye Disease' in model_name or 'ViT' in model_name:
        optimizer = AdamW(learning_rate=1e-4, weight_decay=1e-4)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
    )
    
    # Simple callbacks without ReduceLROnPlateau to avoid conflicts
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,  # Increased patience since we removed ReduceLROnPlateau
            restore_best_weights=True,
            verbose=1
        )
    ]
    
    # Train model
    history = model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=40,
        verbose=1
    )
    
    # Plot individual training history
    plot_clean_training_history(history, model_name)
    
    return model, history

def measure_inference_time(model, test_gen, num_batches=10):
    """Measure inference time"""
    times = []
    
    for i, (x_batch, y_batch) in enumerate(test_gen):
        if i >= num_batches:
            break
        
        start_time = time.time()
        _ = model.predict(x_batch, verbose=0)
        end_time = time.time()
        
        times.append(end_time - start_time)
    
    avg_time_per_batch = np.mean(times)
    avg_time_per_image = avg_time_per_batch / batch_size
    
    return avg_time_per_batch, avg_time_per_image

def plot_combined_training_histories(histories):
    """Combined training histories visualization"""
    plt.figure(figsize=(20, 12))
    plt.style.use('fivethirtyeight')
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']
    
    # Training Loss
    plt.subplot(2, 2, 1)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['loss']))]
        plt.plot(epochs, history.history['loss'], 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
    plt.title('Training Loss Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Validation Loss
    plt.subplot(2, 2, 2)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['val_loss']))]
        val_loss = history.history['val_loss']
        plt.plot(epochs, val_loss, 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
        
        # Mark best epoch
        best_epoch = np.argmin(val_loss) + 1
        best_loss = min(val_loss)
        plt.scatter(best_epoch, best_loss, 
                   color=colors[idx % len(colors)], s=100, zorder=5)
        
    plt.title('Validation Loss Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Training Accuracy
    plt.subplot(2, 2, 3)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['accuracy']))]
        plt.plot(epochs, history.history['accuracy'], 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
    plt.title('Training Accuracy Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Validation Accuracy
    plt.subplot(2, 2, 4)
    for idx, (model_name, history) in enumerate(histories.items()):
        epochs = [i+1 for i in range(len(history.history['val_accuracy']))]
        val_acc = history.history['val_accuracy']
        plt.plot(epochs, val_acc, 
                color=colors[idx % len(colors)], linewidth=2, label=model_name)
        
        # Mark best epoch
        best_epoch = np.argmax(val_acc) + 1
        best_acc = max(val_acc)
        plt.scatter(best_epoch, best_acc, 
                   color=colors[idx % len(colors)], s=100, zorder=5)
        
    plt.title('Validation Accuracy Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_comprehensive_results(results):
    """Comprehensive results visualization with 4-digit precision"""
    fig, axes = plt.subplots(3, 3, figsize=(20, 18))
    plt.style.use('default')
    
    models = list(results.keys())
    colors = plt.cm.Set3(np.linspace(0, 1, len(models)))
    
    # Test Accuracy Comparison
    test_accs = [results[model]['test_accuracy'] for model in models]
    bars1 = axes[0, 0].bar(models, test_accs, color=colors)
    axes[0, 0].set_title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(test_accs):
        axes[0, 0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Parameters vs Accuracy
    params = [results[model]['parameters'] / 1e6 for model in models]
    scatter = axes[0, 1].scatter(params, test_accs, s=150, c=colors, alpha=0.7)
    axes[0, 1].set_title('Parameters vs Test Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Parameters (Millions)')
    axes[0, 1].set_ylabel('Test Accuracy')
    for i, model in enumerate(models):
        axes[0, 1].annotate(model, (params[i], test_accs[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # Inference Time Comparison
    inf_times = [results[model]['inference_time_per_image'] * 1000 for model in models]
    bars2 = axes[0, 2].bar(models, inf_times, color=colors)
    axes[0, 2].set_title('Inference Time per Image', fontsize=14, fontweight='bold')
    axes[0, 2].set_ylabel('Time (ms)')
    axes[0, 2].tick_params(axis='x', rotation=45)
    for i, v in enumerate(inf_times):
        axes[0, 2].text(i, v + max(inf_times)*0.02, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # F1-Score Comparison
    f1_scores = [results[model]['f1_macro'] for model in models]
    bars3 = axes[1, 0].bar(models, f1_scores, color=colors)
    axes[1, 0].set_title('F1-Score (Macro Average)', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('F1-Score')
    axes[1, 0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(f1_scores):
        axes[1, 0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Mean Average Precision
    mean_aps = [results[model]['mean_ap'] for model in models]
    bars4 = axes[1, 1].bar(models, mean_aps, color=colors)
    axes[1, 1].set_title('Mean Average Precision', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('mAP')
    axes[1, 1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(mean_aps):
        axes[1, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Precision vs Recall
    precisions = [results[model]['precision_macro'] for model in models]
    recalls = [results[model]['recall_macro'] for model in models]
    scatter = axes[1, 2].scatter(recalls, precisions, s=150, c=colors, alpha=0.7)
    axes[1, 2].set_title('Precision vs Recall', fontsize=14, fontweight='bold')
    axes[1, 2].set_xlabel('Recall (Macro)')
    axes[1, 2].set_ylabel('Precision (Macro)')
    for i, model in enumerate(models):
        axes[1, 2].annotate(model, (recalls[i], precisions[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # Efficiency Score (Accuracy / Parameters)
    efficiency = [results[model]['test_accuracy'] / (results[model]['parameters'] / 1e6) for model in models]
    bars5 = axes[2, 0].bar(models, efficiency, color=colors)
    axes[2, 0].set_title('Efficiency Score (Acc/Params)', fontsize=14, fontweight='bold')
    axes[2, 0].set_ylabel('Efficiency Score')
    axes[2, 0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(efficiency):
        axes[2, 0].text(i, v + max(efficiency)*0.02, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Top-2 Accuracy
    top2_accs = [results[model]['test_top2_accuracy'] for model in models]
    bars6 = axes[2, 1].bar(models, top2_accs, color=colors)
    axes[2, 1].set_title('Top-2 Accuracy', fontsize=14, fontweight='bold')
    axes[2, 1].set_ylabel('Top-2 Accuracy')
    axes[2, 1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(top2_accs):
        axes[2, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Speed vs Accuracy Trade-off
    axes[2, 2].scatter(inf_times, test_accs, s=150, c=colors, alpha=0.7)
    axes[2, 2].set_title('Speed vs Accuracy Trade-off', fontsize=14, fontweight='bold')
    axes[2, 2].set_xlabel('Inference Time (ms)')
    axes[2, 2].set_ylabel('Test Accuracy')
    for i, model in enumerate(models):
        axes[2, 2].annotate(model, (inf_times[i], test_accs[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.tight_layout()
    plt.show()

# Main execution: Train models one by one, with Eye Disease Classifier at the end
print("\n" + "="*80)
print("SEQUENTIAL MODEL TRAINING - EYE DISEASE CLASSIFICATION")
print("="*80)

# Define models in order (Eye Disease Classifier will be trained last)
models_to_train = [
    ('EfficientNet-B0', create_efficientnet),
    ('MobileNet-V2', create_mobilenet_v2), 
    ('VGG16', create_vgg16),
    ('DenseNet121', create_denseNet201)]

# Store results
results = {}
histories = {}
trained_models = {}

# Train each model sequentially
for model_name, model_creator in models_to_train:
    print(f"\n{'='*60}")
    print(f"CREATING AND TRAINING: {model_name}")
    print(f"{'='*60}")
    
    # Create the model
    model = model_creator(num_classes)
    
    # Build the model using a real batch from train_gen if not built
    if not model.built:
        x_batch, y_batch = next(iter(train_gen))
        _ = model(x_batch)
    
    print(f"Parameters: {model.count_params():,}")
    
    # Reset generators before training
    train_gen.reset()
    valid_gen.reset()
    
    # Train model
    trained_model, history = compile_and_train_model(
        model, model_name, train_gen, valid_gen, epochs=30
    )
    
    # Store trained model and history
    trained_models[model_name] = trained_model
    histories[model_name] = history
    
    # Evaluate on test set
    test_gen.reset()
    test_loss, test_acc, test_top2_acc = trained_model.evaluate(test_gen, verbose=0)
    
    # Get predictions for confusion matrix and classification report
    test_gen.reset()
    y_pred_probs = trained_model.predict(test_gen, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    
    # Get true labels
    y_true = test_gen.classes
    
    # Measure inference time
    test_gen.reset()
    avg_batch_time, avg_image_time = measure_inference_time(trained_model, test_gen)
    
    # Plot confusion matrix with 4-digit precision
    cm = plot_confusion_matrix(y_true, y_pred_classes, class_names, model_name)
    
    # Generate classification report with 4-digit precision
    class_report = generate_classification_report_4digits(
        y_true, y_pred_classes, class_names, model_name
    )
    
    # Plot GradCAM examples (for CNN-based models)
    if any(keyword in model_name for keyword in ['Eye Disease', 'EfficientNet', 'MobileNet', 'VGG', 'DenseNet201', 'MobileNet_v2']):
        plot_gradcam_examples(trained_model, test_gen, class_names, model_name)
        
    
    # Calculate precision-recall curves
    y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes)
    
    # Calculate mean AP with 4-digit precision
    mean_ap = np.mean([average_precision_score(y_true_onehot[:, i], y_pred_probs[:, i]) 
                      for i in range(len(class_names))])
    
    # Store results with 4-digit precision
    results[model_name] = {
        'test_accuracy': round(test_acc, 4),
        'test_top2_accuracy': round(test_top2_acc, 4),
        'test_loss': round(test_loss, 4),
        'parameters': model.count_params(),
        'best_val_accuracy': round(max(history.history['val_accuracy']), 4),
        'mean_ap': round(mean_ap, 4),
        'precision_macro': round(class_report['macro avg']['precision'], 4),
        'recall_macro': round(class_report['macro avg']['recall'], 4),
        'f1_macro': round(class_report['macro avg']['f1-score'], 4),
        'inference_time_per_image': round(avg_image_time, 6),
        'inference_time_per_batch': round(avg_batch_time, 4)
    }
    
    print(f"\n{'='*50}")
    print(f"TEST RESULTS FOR {model_name}:")
    print(f"{'='*50}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Top-2 Accuracy: {test_top2_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Mean AP: {mean_ap:.4f}")
    print(f"F1-Score (Macro): {class_report['macro avg']['f1-score']:.4f}")
    print(f"Precision (Macro): {class_report['macro avg']['precision']:.4f}")
    print(f"Recall (Macro): {class_report['macro avg']['recall']:.4f}")
    print(f"Inference time per image: {avg_image_time*1000:.4f} ms")
    print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")
    print(f"Parameters: {model.count_params():,}")
    
    # Clean up to save memory (optional)
    if model_name != 'Eye Disease Classifier':  # Keep your custom model
        del model

# Combined visualizations after all models are trained
print(f"\n{'='*80}")
print("COMBINED TRAINING HISTORIES VISUALIZATION")
print(f"{'='*80}")
plot_combined_training_histories(histories)

# Comprehensive results visualization
print(f"\n{'='*80}")
print("COMPREHENSIVE RESULTS VISUALIZATION")
print(f"{'='*80}")
plot_comprehensive_results(results)

# Print final detailed comparison with 4-digit precision
print("\n" + "="*130)
print("COMPREHENSIVE RESULTS COMPARISON (4-DIGIT PRECISION)")
print("="*130)

results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('test_accuracy', ascending=False)

print(f"{'Model':<20} {'Test Acc':<10} {'Top-2 Acc':<10} {'mAP':<10} {'F1':<10} {'Precision':<10} {'Recall':<10} {'Params(M)':<12} {'Time(ms)':<12}")
print("-" * 130)

for model_name, row in results_df.iterrows():
    print(f"{model_name:<20} "
          f"{row['test_accuracy']:<10.4f} "
          f"{row['test_top2_accuracy']:<10.4f} "
          f"{row['mean_ap']:<10.4f} "
          f"{row['f1_macro']:<10.4f} "
          f"{row['precision_macro']:<10.4f} "
          f"{row['recall_macro']:<10.4f} "
          f"{row['parameters']/1e6:<12.2f} "
          f"{row['inference_time_per_image']*1000:<12.4f}")

# Final analysis and recommendations
print("\n" + "="*130)
print("FINAL ANALYSIS AND RECOMMENDATIONS")
print("="*130)

best_model = results_df.index[0]
efficiency_scores = results_df['test_accuracy'] / (results_df['parameters'] / 1e6)
most_efficient = efficiency_scores.idxmax()
fastest = results_df['inference_time_per_image'].idxmin()

print(f"🏆 BEST OVERALL PERFORMANCE: {best_model}")
print(f"   → Test Accuracy: {results_df.loc[best_model, 'test_accuracy']:.4f}")
print(f"   → F1-Score: {results_df.loc[best_model, 'f1_macro']:.4f}")
print(f"   → mAP: {results_df.loc[best_model, 'mean_ap']:.4f}")
print(f"   → Parameters: {results_df.loc[best_model, 'parameters']/1e6:.2f}M")

print(f"\n⚡ MOST EFFICIENT: {most_efficient}")
print(f"   → Efficiency Score: {efficiency_scores[most_efficient]:.4f}")
print(f"   → Test Accuracy: {results_df.loc[most_efficient, 'test_accuracy']:.4f}")
print(f"   → Parameters: {results_df.loc[most_efficient, 'parameters']/1e6:.2f}M")

print(f"\n🚀 FASTEST INFERENCE: {fastest}")
print(f"   → Inference Time: {results_df.loc[fastest, 'inference_time_per_image']*1000:.4f} ms")
print(f"   → Test Accuracy: {results_df.loc[fastest, 'test_accuracy']:.4f}")
print(f"   → FPS: {1/results_df.loc[fastest, 'inference_time_per_image']:.1f}")

# Eye Disease Classifier specific analysis
if 'Eye Disease Classifier' in results_df.index:
    eye_model_results = results_df.loc['Eye Disease Classifier']
    print(f"\n🔬 YOUR EYE DISEASE CLASSIFIER ANALYSIS:")
    print(f"   → Test Accuracy: {eye_model_results['test_accuracy']:.4f}")
    print(f"   → F1-Score: {eye_model_results['f1_macro']:.4f}")
    print(f"   → Mean AP: {eye_model_results['mean_ap']:.4f}")
    print(f"   → Efficiency: {efficiency_scores['Eye Disease Classifier']:.4f}")
    print(f"   → Inference Time: {eye_model_results['inference_time_per_image']*1000:.4f} ms")
    print(f"   → Rank by Accuracy: {list(results_df.index).index('Eye Disease Classifier') + 1}/{len(results_df)}")

# Save comprehensive results
results_df.to_csv('eye_disease_comprehensive_results.csv')
print(f"\n📊 Results saved to: eye_disease_comprehensive_results.csv")
print(f"\n{'='*130}")
print("ANALYSIS COMPLETE - ALL MODELS TRAINED SEQUENTIALLY")
print(f"{'='*130}")


In [ ]:
trained_models["MobileNet-V2"].save("mobilenetv2_deployment_model.h5")


In [ ]:
import tensorflow as tf

# Get trained MobileNetV2 model
mobilenet_model = trained_models['MobileNet-V2']

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save TFLite model
with open("mobilenetv2_eye_disease.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ MobileNetV2 TFLite model saved successfully")


In [ ]:
!ls -lh /kaggle/working/